## We need stable diffusion model and docker image in the EC2 instance

In [ ]:
# permission to use sagemaker including 
# AmazonEC2ContainerRegistryFullAccess, AmazonS3FullAccess, AmazonSageMakerFullAccess
import sagemaker 
import boto3  
sess = sagemaker.Session() 
# sagemaker session bucket -> used for uploading data, models and logs 
# sagemaker will automatically create this bucket if it not exists 
sagemaker_session_bucket = None 
if sagemaker_session_bucket is None and sess is not None:     
    # set to default bucket if a bucket name is not given  
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role() 
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)  
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}") 
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
# go to test_dir directory
%cd test_dir/

## Upload your images to a new folder e.g. zwx (zwx is your instance class)
#### ./local_test/test_dir/input/data/

In [ ]:
# remove the old model if exists
# if no model, skip this step
# rm model and pic
!sudo rm -rf ./model
# make model directory 
!mkdir -p model

In [ ]:
MODEL_NAME = "./stable-diffusion-v1-5"
OUTPUT_DIR = "./model"
ACCELERATE_CONFIG = "./accelerate-config.yaml"

In [ ]:
# need to update the instance prompt and class prompt folder info
# concepts_list.json is generated
local_instance_dir = "./input/data/zwx"

concepts_list = [
    {
        "instance_prompt":      "photo of zwx model",
        "class_prompt":         "photo of a model",
        "instance_data_dir":    "/opt/ml/input/data/zwx",
        "class_data_dir":       "/opt/ml/input/data/woman"
    }
]

import json
import os
os.makedirs(local_instance_dir, exist_ok=True)

with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)

In [ ]:
# model config
ACCELERATE_CONFIG="/opt/ml/accelerate-config.yaml"
MODEL_NAME="/opt/ml/stable-diffusion-v1-5"
OUTPUT_DIR="/opt/ml/model"
CONCEPT_LIST="/opt/ml/concepts_list.json"

In [ ]:
# go to local_test directory to run train_dreamscrape.py
%cd ..

In [ ]:
!sudo docker run -it --gpus=all --ipc=host -v $(pwd)/test_dir:/opt/ml --rm pytorch-sagemaker-training:v1.0 \
  accelerate launch --config_file=$ACCELERATE_CONFIG /opt/ml/train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --revision="fp16" \
  --with_prior_preservation=True \
  --prior_loss_weight=1.0 \
  --seed=15213 \
  --resolution=512 \
  --train_batch_size=2 \
  --train_text_encoder=True \
  --mixed_precision="fp16" \
  --use_8bit_adam=True \
  --gradient_accumulation_steps=1 \
  --gradient_checkpointing=True \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=50 \
  --sample_batch_size=4 \
  --max_train_steps=1500 \
  --save_interval=10000 \
  --concepts_list=$CONCEPT_LIST 
        
        

In [ ]:
%cd model

In [ ]:
# upload the model to s3 bucket
import boto3
bucket = "S3_BUCKET"
s3_bucket_dir = "MODEL_STORAGE_KEY"
s3 = boto3.resource('s3')
s3.meta.client.upload_file('model.tar.gz', bucket, s3_bucket_dir)


In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel

def deploy_huggingface_sagemaker(model_s3_location, role):
    huggingface_model = HuggingFaceModel(
        model_data=model_s3_location,  # path to your model & script
        role=role,  # iam role with permissions for Endpoint
        transformers_version="4.17",  # transformers version used
        pytorch_version="1.10",  # pytorch version used
        py_version="py38",  # python version used
    )

    # deploy the endpoint endpoint
    predictor = huggingface_model.deploy(
        initial_instance_count=1, 
        instance_type="ml.g4dn.xlarge"
    )
    return predictor.endpoint_name

In [ ]:
endpoint = deploy_huggingface_sagemaker(model_path, role)